In [1]:
import torch
from torch.utils.data import DataLoader
import torch as nn
from torch import nn
import torch.optim as optim
import numpy as np
from sklearn.metrics import roc_auc_score, accuracy_score, matthews_corrcoef, recall_score, precision_score
from sklearn.metrics import confusion_matrix, f1_score, classification_report
from sklearn.metrics import roc_curve
import torch.nn.functional as F
from torchvision import datasets, models

In [2]:
loaded_datasets_info = torch.load('/root/autodl-tmp/data/saved_datasets.pth', weights_only=False)
train_dataset = loaded_datasets_info['train_dataset']
test_dataset = loaded_datasets_info['test_dataset']

In [3]:
batch_size = 100
loaded_train_dataset = DataLoader(train_dataset, batch_size = batch_size, shuffle = False)
loaded_test_dataset = DataLoader(test_dataset, batch_size = batch_size, shuffle = False)

In [4]:
class SimpleGoogLeNet(nn.Module):
    def __init__(self, num_classes=10):
        super(SimpleGoogLeNet, self).__init__()
 
        # 第一个 Inception 模块
        self.inception1 = nn.Sequential(
            nn.Conv2d(1, 32, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2)
        )
 
        # 第二个 Inception 模块
        self.inception2 = nn.Sequential(
            nn.Conv2d(32, 64, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2)
        )
 
        # 第三个 Inception 模块
        self.inception3 = nn.Sequential(
            nn.Conv2d(64, 128, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2)
        )
 
        # 分类器：全连接层 + Dropout 层
        self.fc = nn.Sequential(
            nn.Linear(128 * 3 * 3, 1024),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(1024, num_classes)
        )
 
    def forward(self, x):
        x = self.inception1(x)
        x = self.inception2(x)
        x = self.inception3(x)
        x = x.view(x.size(0), -1)  # 展平输入
        x = self.fc(x)
        return x

In [5]:
device = "cuda"
model = SimpleGoogLeNet().to(device)
criterion = nn.CrossEntropyLoss().to(device)
optimizer = torch.optim.SGD(model.parameters(), lr=0.001, momentum=0.9)
num_epochs = 10

In [6]:
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for batch_indx, (inputs, labels) in enumerate(loaded_train_dataset):
        inputs = inputs.to(device)
        labels = labels.to(device)
        
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
        
    # Print average loss for the epoch
    print(f"Epoch {epoch+1}/{num_epochs}, Loss: {running_loss / (len(loaded_train_dataset) / batch_size)}")  

Epoch 1/10, Loss: 23.606188830376293
Epoch 2/10, Loss: 6.702565330449336
Epoch 3/10, Loss: 4.699333260272397
Epoch 4/10, Loss: 3.734402134733197
Epoch 5/10, Loss: 3.091288264595884
Epoch 6/10, Loss: 2.5414104811716243
Epoch 7/10, Loss: 2.043647158867922
Epoch 8/10, Loss: 1.8662957793815924
Epoch 9/10, Loss: 1.665732784280408
Epoch 10/10, Loss: 1.4267580561184634


In [7]:
torch.save(model.state_dict(), './model_params/GoogLeNet.pth')

In [8]:
model.load_state_dict(torch.load('./model_params/GoogLeNet.pth'))

/tmp/ipykernel_1463/193070061.py:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load('./model_params/GoogLeNet.pth'))


<All keys matched successfully>

### Metrics function definition

In [24]:
# get roc_auc, metrics_sn, metrics_sp, metrics_ACC, metrics_F1, metrics_MCC
def calculate_multiclass_metrics(true_labels, predicted_labels, predicted_probabilities, num_classes):
    accuracy = accuracy_score(true_labels, predicted_labels)
    mcc = matthews_corrcoef(true_labels, predicted_labels)
    
    sensitivity_per_class = []
    specificity_per_class = []
    auc_per_class = []
    f1_per_class = []

    for i in range(num_classes):
        true_binary = (np.array(true_labels) == i).astype(int)
        pred_binary = (np.array(predicted_labels) == i).astype(int)

        cm = confusion_matrix(true_binary, pred_binary, labels=[0, 1])
        tn, fp, fn, tp = cm.ravel()

        sensitivity = tp / (tp + fn) if (tp + fn) > 0 else 0
        specificity = tn / (tn + fp) if (tn + fp) > 0 else 0
        sensitivity_per_class.append(sensitivity)
        specificity_per_class.append(specificity)

        
        auc = roc_auc_score(true_binary, predicted_probabilities[:, i]) if len(np.unique(true_binary)) > 1 else 0
        auc_per_class.append(auc)

        f1 = f1_score(true_binary, pred_binary) if len(np.unique(true_binary)) > 1 else 0
        f1_per_class.append(f1)

    avg_sensitivity = np.mean(sensitivity_per_class)
    avg_specificity = np.mean(specificity_per_class)
    avg_auc = np.mean(auc_per_class) if auc_per_class else 0
    avg_f1 = np.mean(f1_per_class)

    print(f"Average AUC: {avg_auc:.4f}")
    print(f"Average Sensitivity: {avg_sensitivity:.4f}")
    print(f"Average Specificity: {avg_specificity:.4f}")
    print(f"Accuracy: {accuracy:.4f}")
    print(f"Average F1-score: {avg_f1:.4f}")
    print(f"Matthews Correlation Coefficient (MCC): {mcc:.4f}")

### Training data metrics

In [25]:
predicted_probabilities = []
true_labels = []
with torch.set_grad_enabled(False):
    for batch_indx, (inputs, labels) in enumerate(loaded_train_dataset):
        inputs = inputs.to(device)
        labels = labels.to(device)      
        outputs = model(inputs)
        predicted_probabilities.extend(outputs.tolist())
        true_labels.extend(labels.tolist())

In [26]:
true_labels_ = np.argmax(true_labels, axis=-1)
predicted_labels = np.argmax(predicted_probabilities, axis=-1)
preds = torch.tensor(predicted_probabilities)
preds = F.softmax(preds, dim=-1)

train_metrics = calculate_multiclass_metrics(true_labels_, predicted_labels, preds, num_classes=10)

Average AUC: 1.0000
Average Sensitivity: 0.9957
Average Specificity: 0.9995
Accuracy: 0.9957
Average F1-score: 0.9957
Matthews Correlation Coefficient (MCC): 0.9952


In [29]:
np.save('/root/autodl-tmp/ROC/GoogLeNet/y_train_pred.npy', preds)
np.save('/root/autodl-tmp/ROC/GoogLeNet/y_train.npy', true_labels)

### Testing data metrics

In [30]:
predicted_probabilities = []  
true_labels = []  
with torch.set_grad_enabled(False): 
    for batch_indx, (inputs, labels) in enumerate(loaded_test_dataset):
        inputs = inputs.to(device)
        labels = labels.to(device)    
        outputs = model(inputs)
        predicted_probabilities.extend(outputs.tolist())
        true_labels.extend(labels.tolist())

In [31]:
true_labels_ = np.argmax(true_labels, axis=-1)
predicted_labels = np.argmax(predicted_probabilities, axis=-1)
preds = torch.tensor(predicted_probabilities)
preds = F.softmax(preds, dim=-1)

test_metrics = calculate_multiclass_metrics(true_labels_, predicted_labels, preds, num_classes=10)

Average AUC: 0.9999
Average Sensitivity: 0.9891
Average Specificity: 0.9988
Accuracy: 0.9891
Average F1-score: 0.9890
Matthews Correlation Coefficient (MCC): 0.9879


In [32]:
np.save('/root/autodl-tmp/ROC/GoogLeNet/y_test_pred.npy', preds)
np.save('/root/autodl-tmp/ROC/GoogLeNet/y_test.npy', true_labels)